In [ ]:
!pip install noisereduce

In [ ]:
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
import librosa.display
import folium
from folium import plugins
import torch
import torchaudio
from math import ceil
import noisereduce as nr
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('../input/birdclef-2022/train_metadata.csv')
train[0:3].T

In [ ]:
train_audio_dir='../input/birdclef-2022/train_audio'

In [ ]:
paths=[]
for i in range(len(train)):
    paths+=[os.path.join(train_audio_dir,train.loc[i,'filename'])]

In [ ]:
def create_spectrogram(
    fname: str,
    reduce_noise: bool = False,
    frame_size: int = 5,
    frame_step: int = 2,
    channel: int = 0,
    device = "cpu",
) -> list:
    waveform, sample_rate = torchaudio.load(fname)
    
    transform = torchaudio.transforms.Spectrogram(n_fft=1800, win_length=512).to(device)
    if reduce_noise:
        waveform = torch.tensor(nr.reduce_noise(
            y=waveform,
            sr=sample_rate,
            win_length=transform.win_length,
            use_tqdm=False,
            n_jobs=2,
        ))
    step = int(frame_step * sample_rate)
    size = int(frame_size * sample_rate)
    spectrograms = []
    for i in range(ceil((waveform.size()[-1] - size) / step)):
        begin = i * step
        frame = waveform[channel][begin:begin + size]
        if len(frame) < size:
            if i == 0:
                rep = round(float(size) / len(frame))
                frame = frame.repeat(int(rep))
            elif len(frame) < (size * 0.33):
                continue
            else:
                frame = waveform[channel][-size:]
        sg = transform(frame.to(device))
        spectrograms.append(np.nan_to_num(torch.log(sg).numpy()))
        # spectrograms.append(np.nan_to_num(sg.numpy()))
    return spectrograms

In [ ]:
i=8 #set number
display(train.iloc[i])
print()
print(paths[i])
print()
ipd.Audio(paths[i])

In [ ]:
path_audio = paths[i]
sgs = create_spectrogram(path_audio, reduce_noise=True)
print(len(sgs))

fig, axarr = plt.subplots(ncols=4, figsize=(16,4))
for i, sg in enumerate(sgs[0:4]):
    ax = axarr[i].imshow(sg, vmin=-50, vmax=10)
    
plt.colorbar(ax)
plt.show()

In [ ]:
data=train.iloc[i:i+1,3:5]
eq_map = folium.Map(location=[20,0],tiles='Stamen Terrain',zoom_start=1,min_zoom=2.0)
eq_map.add_child(plugins.HeatMap(data))
eq_map